# Summary
1. Here, we explore how to create unnatural bareground layers. We will consider annual rainfal and classify pixels that are not green enough as bare ground.

### Future
-

# Setting up the environment

In [ ]:
# Import and/or install libraries
import subprocess, os, gcsfs, json

try:
    import geemap, ee
except ImportError:
    subprocess.check_call(["python", '-m', 'pip', 'install', '-U', 'geemap'])
    import geemap, ee


In [ ]:
# Connect to Google Drive to access files

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Connect to Google Earth Engine if neccessary

service_account = os.environ.get('GOOGLE_SERVICE_ACCOUNT')
credentials = ee.ServiceAccountCredentials(service_account, os.environ.get('GOOGLE_APPLICATION_CREDENTIALS'))
ee.Initialize(credentials)

In [ ]:
Map = geemap.Map()
Map.add_basemap('SATELLITE')
Map.setCenter(31.944, -25.087, 12)
Map